# Aerosol Prediction Neural Network

## Data Caracterization

### Data loading

In [19]:
import pandas as pd

# Load the dataset
df = pd.read_csv("datasets/train.csv")

print(f'Number of features: {df.shape[1]}')
print(f'Number of instances: {df.shape[0]}')
df.head()

Number of features: 10
Number of instances: 10438


,id,elevation,ozone,NO2,azimuth,zenith,incidence_azimuth,incidence_zenith,file_name_l1,value_550
0,1,10,318,0.248,150.6,31.8,286.1,8.0,AAOT_45-3139_12-5083_COPERNICUS_S2_20180807T10...,0.277
1,2,10,302,0.279,161.6,44.2,243.6,3.9,AAOT_45-3139_12-5083_COPERNICUS_S2_20180916T10...,0.201
2,4,10,373,0.303,163.5,34.4,103.9,9.8,AAOT_45-3139_12-5083_COPERNICUS_S2_20190421T10...,0.169
3,5,10,342,0.271,144.7,25.3,286.2,7.9,AAOT_45-3139_12-5083_COPERNICUS_S2_20190623T10...,0.107
4,6,10,327,0.252,140.4,29.4,105.8,7.0,AAOT_45-3139_12-5083_COPERNICUS_S2_20190720T10...,0.188


As we can see the dataset is composed of 10 features and has 10438 instances or observations.

The target feature is the 'value_550', the one we want to be capable of predicting.

Then we can split the features into two groups, the numeric features, and the image feature. The features, 'id' (identification feature, is not important for the training and prediction), 'elevation', 'ozone', 'NO2', 'azimuth', 'zenith', 'incidence_azimuth', and 'incident_zenith' are the numeric features, that will be scaled for a better Neural Network Model training. At last, but not least, the feature file_name_l1 is the name associated to the image from the zone where the other features where measured.

## Data preprocessing

### Data separation (numerical and images)

In [20]:
from sklearn.preprocessing import StandardScaler
from keras.api.utils import load_img, img_to_array
import numpy as np
import os
import tifffile as tiff

# Preprocess numerical data
numerical_features = df[['elevation', 'ozone', 'NO2', 'azimuth', 'zenith', 'incidence_azimuth', 'incidence_zenith']]

# Numerical data scaling
scaler = StandardScaler()
numerical_features = scaler.fit_transform(numerical_features)

# Function to load and preprocess image data
def load_and_preprocess_image(filepath):
    img = tiff.imread(filepath)
    img_array = np.array(img)
    img_array = img_array / 65535.0   # Normalize pixel values
    return img_array

# Load image data
image_data = np.array([load_and_preprocess_image(os.path.join('./train/', filename)) for filename in df['file_name_l1']])

### Train and Validation Split

In [21]:
from sklearn.model_selection import train_test_split

# Target variable
target = df['value_550'].values

# Split data into training, validation, and testing sets
X_train_num, X_temp_num, X_train_img, X_temp_img, y_train, y_temp = train_test_split(numerical_features, image_data, target, test_size=0.3, random_state=42)
X_val_num, X_test_num, X_val_img, X_test_img, y_val, y_test = train_test_split(X_temp_num, X_temp_img, y_temp, test_size=0.5, random_state=42)

## Neural Network Arquitecture

In [24]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Input, Concatenate, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2

# Define the CNN and dense model
class AOTModel:
    def __init__(self, image_shape=(19, 19, 13), num_numerical_features=7):
        # Image processing Neural Network
        self.image_input = Input(shape=image_shape)
        # image_processing_network = Conv2D(32, (3, 3), activation='relu')(self.image_input)
        # image_processing_network = BatchNormalization()(image_processing_network)
        # image_processing_network = MaxPooling2D((2, 2))(image_processing_network)
        # image_processing_network = Dropout(0.25)(image_processing_network)

        # image_processing_network = Conv2D(64, (3, 3), activation='relu')(image_processing_network)
        # image_processing_network = BatchNormalization()(image_processing_network)
        # image_processing_network = MaxPooling2D((2, 2))(image_processing_network)
        # image_processing_network = Dropout(0.25)(image_processing_network)

        # image_processing_network = Flatten()(image_processing_network)
        # image_processing_network = Dense(64, activation='relu')(image_processing_network)
        # image_processing_network = Dropout(0.5)(image_processing_network)
        
        image_processing_network = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01))(self.image_input)
        image_processing_network = BatchNormalization()(image_processing_network)
        image_processing_network = MaxPooling2D((2, 2))(image_processing_network)
        image_processing_network = Dropout(0.25)(image_processing_network)

        image_processing_network = Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01))(image_processing_network)
        image_processing_network = BatchNormalization()(image_processing_network)
        image_processing_network = MaxPooling2D((2, 2))(image_processing_network)
        image_processing_network = Dropout(0.25)(image_processing_network)

        image_processing_network = Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01))(image_processing_network)
        image_processing_network = BatchNormalization()(image_processing_network)
        image_processing_network = GlobalAveragePooling2D()(image_processing_network)
        image_processing_network = Dropout(0.5)(image_processing_network)

        # Numerical processing Neural Network
        self.numerical_input = Input(shape=(num_numerical_features,))
        numerical_processing_network = Dense(64, activation='relu')(self.numerical_input)
        numerical_processing_network = BatchNormalization()(numerical_processing_network)
        numerical_processing_network = Dropout(0.5)(numerical_processing_network)
        
        # Concatenation of both networks
        aot_network = Concatenate()([image_processing_network, numerical_processing_network])
        aot_network = Dense(64, activation='relu')(aot_network)
        aot_network = Dropout(0.5)(aot_network)
        aot_network = Dense(1)(aot_network)

        self.aot_network_arquitecture = aot_network
        del image_processing_network, numerical_processing_network, aot_network

    def model(self):
        model = Model(inputs= [self.image_input, self.numerical_input], outputs=self.aot_network_arquitecture)
        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
        return model


## Training

In [25]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Instantiate the model
model = AOTModel()
model = model.model()

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Data augmentation
# datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=10)

datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    zoom_range=0.2
)

# Train the model
history = model.fit(
    [X_train_img, X_train_num], y_train,
    validation_data=([X_val_img, X_val_num], y_val),
    epochs=200,
    # batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

model.save('aot_model.h5')

# Evaluate the model
val_loss, val_mae = model.evaluate([X_val_img, X_val_num], y_val)
print(f'Validation MAE: {val_mae}')

# Evaluate on test set
test_loss, test_mae = model.evaluate([X_test_img, X_test_num], y_test)
print(f'Test MAE: {test_mae}')

Epoch 1/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 31s 66ms/step - loss: 5.2631 - mae: 1.4030 - val_loss: 1.4092 - val_mae: 0.1962 - learning_rate: 0.0010
Epoch 2/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 2.0385 - mae: 0.6496 - val_loss: 1.2222 - val_mae: 0.1464 - learning_rate: 0.0010
Epoch 3/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.4104 - mae: 0.4012 - val_loss: 1.0386 - val_mae: 0.1168 - learning_rate: 0.0010
Epoch 4/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.0955 - mae: 0.2764 - val_loss: 0.8705 - val_mae: 0.1120 - learning_rate: 0.0010
Epoch 5/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.8840 - mae: 0.2162 - val_loss: 0.7135 - val_mae: 0.1007 - learning_rate: 0.0010
Epoch 6/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.7017 - mae: 0.1708 - val_loss: 0.5756 - val_mae: 0.0934 - learning_rate: 0.0010
Epoch 7/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.5559 - mae: 0.1416 - val_loss: 0.4575 - val_mae: 0.0870 - learning_rate: 0.00

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0143 - mae: 0.0695
Validation MAE: 0.06951191276311874
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0120 - mae: 0.0657
Test MAE: 0.06991945207118988


## Predictions

In [26]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('aot_model.h5')

# Load the new dataset
new_df = pd.read_csv("datasets/test.csv")

# Preprocess numerical data
new_numerical_features = new_df[['elevation', 'ozone', 'NO2', 'azimuth', 'zenith', 'incidence_azimuth', 'incidence_zenith']]
new_numerical_features = scaler.transform(new_numerical_features)

# Load and preprocess new image data
new_image_data = np.array([load_and_preprocess_image(os.path.join('./test/', filename)) for filename in new_df['file_name_l1']])

# Predict values for the new data
predictions = model.predict([new_image_data, new_numerical_features])

# Save the predictions to a CSV file
results = pd.DataFrame({
    'id': new_df['id'],
    'value_550': predictions.flatten()
})
results.to_csv('predictions.csv', index=False)

print(results.head())

85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
   id  value_550
0   3   0.180006
1  25   0.213587
2  26   0.096308
3  27   0.126866
4  29   0.104977
